In [11]:
from pathlib import Path
import pickle
from joblib import Parallel, delayed
from itertools import product
from smount_predictors.src.SeamountHelp import PipelinePredictor
from smount_predictors import SeamountHelp
import numpy as np
import pandas as pd
import simplekml
import xarray as xr

In [12]:
import os
os.chdir('interfaces_exclude')

In [13]:
! gmt grdlandmask -I1m -Rswot_masked.grd -Glandmask.grd -N1/0/0/0/0
! gmt grdmath landmask.grd swot_masked.grd MUL = swot_masked_landmask.grd
! mv swot_masked_landmask.grd ../data/

In [14]:
os.chdir('..')

In [15]:
longitude_pairs = []
for lon in range(-180, 180, 5):
    longitude_pairs.append((lon, lon + 5))

latitude_pairs = []
for lat in range(-90, 90, 5):
    latitude_pairs.append((lat, lat + 5))


latlons = list(product(longitude_pairs, latitude_pairs))
latlons = latlons

In [16]:
model = pickle.load(open('out/cluster_tuned_model.pkl', 'rb'))
data_p = Path('data/swot_masked_landmask.grd')

In [17]:
def recluster(model: PipelinePredictor, predictions: pd.DataFrame):
    def filter_clust_size(data: pd.DataFrame):
        def circle_ratio(data: pd.DataFrame):
            if abs(data['lon'].max() - data['lon'].min()) == 0:
                return 0
            if data.loc[:, 'cluster'].mean() == -1:
                return 1
            circle = abs(data['lat'].max() - data['lat'].min()) / abs(data['lon'].max() - data['lon'].min())
            mass = (abs(data['lat'].max() - data['lat'].min()) * abs(data['lon'].max() - data['lon'].min())) / data.shape[0]
            if circle == 0:
                return 0
            mass = 4 * mass / (circle * np.pi)  # Scale mass to be 1 if it is the correct mass of a circle
            return mass
        circle_range = data
        divs = circle_range.groupby('cluster').apply(circle_ratio)
        divs = divs[(divs > np.mean(divs) - np.std(divs)) & (divs < np.mean(divs) + np.std(divs))]
        circle_range = circle_range.loc[(~circle_range['cluster'].isin(divs.index)) & (circle_range['cluster'] != -1)]
        return circle_range

    def norm_z(data: pd.DataFrame):
        data['norm_z'] = (data['z'] - data['z'].min()) / (data['z'].max() - data['z'].min())
        return data
    predictions = predictions.reset_index()
    clust_filt = predictions.copy()

    clust_filt = filter_clust_size(clust_filt)
    clust_filt = norm_z(clust_filt)
    recluster_index = (clust_filt['cluster'] != -1) & (clust_filt['norm_z'] > 0.5)
    clust_pred = clust_filt.loc[recluster_index]


    model.clusterer.fit_predict(clust_pred[['lon', 'lat']])
    clust_filt.loc[recluster_index, 'cluster'] = model.clusterer.labels_ + predictions['cluster'].max() + 1
    clust_filt.loc[~recluster_index, 'cluster'] = -1
    clust_filt.set_index(['lon', 'lat'], inplace=True)
    predictions.set_index(['lon', 'lat'], inplace=True)
    predictions.loc[clust_filt.index, 'cluster'] = clust_filt.loc[:, 'cluster']
    return predictions.reset_index()

In [18]:
def predict_zone(zone):
    lon = zone[0]
    lat = zone[1]
    data = SeamountHelp.readAndFilterGRD(data_p, lat, lon).to_dataframe().reset_index()
    if np.all(data['z'] == 0):
        data['class'] = 0
        data['cluster'] = -1
        return data
    zone_pred = model.predict(data[['lon', 'lat', 'z']])
    return zone_pred

predictions = Parallel(n_jobs=-3)(delayed(predict_zone)(zone) for zone in latlons)
for idx, df in enumerate(predictions):  # ensure non-overlapping cluster numbers
    assert isinstance(df, pd.DataFrame)  # assertion for code linter typing features
    df.loc[df['cluster'] != -1, 'cluster'] = df.loc[df['cluster'] != -1, 'cluster'] + ((idx ** 2) * len(np.unique(df['cluster'])))
predictions = pd.concat(predictions)
predictions = recluster(model, predictions)
predictions['lon'] = np.degrees(predictions['lon'])
predictions['lat'] = np.degrees(predictions['lat'])
global_predictions = xr.Dataset.from_dataframe(predictions).set_coords(['lon', 'lat']).drop('index')
global_predictions.to_netcdf('out/global_predictions.nc')

/Users/m1hawks/Library/Python/3.11/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/m1hawks/Library/Python/3.11/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/m1hawks/Library/Python/3.11/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/m1hawks/Library/Python/3.11/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be cau

In [19]:
mounts = predictions.groupby('cluster').mean().reset_index()
kml = simplekml.Kml()
for i, row in mounts.iterrows():
    kml.newpoint(name=f'{row.cluster}', coords=[(row.lon, row.lat, row.z)])
kml.save('out/global_mounts.kml')

In [20]:
! open out/global_mounts.kml

/var/folders/pv/8l65k1dd4wl0_vtr3mydz87w0160hr/T/ipykernel_65564/2384483886.py:1: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  xr.Dataset.from_dataframe(predictions).set_coords(['lon', 'lat']).drop('index')


<xarray.Dataset> Size: 7GB
Dimensions:  (index: 207360000)
Coordinates:
    lon      (index) float64 2GB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
    lat      (index) float64 2GB -79.99 -79.97 -79.96 ... 79.96 79.97 79.99
Dimensions without coordinates: index
Data variables:
    z        (index) float32 829MB -0.0 -0.0 -0.0 -0.0 ... -4.351 -1.299 -0.9339
    class    (index) int64 2GB 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    cluster  (index) int64 2GB -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1

In [25]:
predictions.set_index(['lat', 'lon'])

,,index,z,class,cluster
lat,lon,,,,
-79.991667,-179.991667,0,-0.000000,0,-1
-79.975000,-179.991667,1,-0.000000,0,-1
-79.958333,-179.991667,2,-0.000000,0,-1
-79.941667,-179.991667,3,-0.000000,0,-1
-79.925000,-179.991667,4,-0.000000,0,-1
...,...,...,...,...,...
79.925000,179.991667,89995,-14.027990,0,-1
79.941667,179.991667,89996,-10.674410,0,-1
79.958333,179.991667,89997,-4.350842,0,-1
